In [2]:
from datasets import load_from_disk

/home/haznitrama/scale-resources/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
cbn_wiki_id = load_from_disk("dataset/paralel_id_cbn_16k")
cbn_wiki_jv = load_from_disk("dataset/paralel_jv_cbn_3k")
cbn_wiki_su = load_from_disk("dataset/paralel_su_cbn_3k")

In [21]:
id_wiki_id = load_from_disk("synthetic/id_titles/id_wiki-id/translated")
id_wiki_jv = load_from_disk("synthetic/id_titles/id_wiki-jv/translated")
id_wiki_su = load_from_disk("synthetic/id_titles/id_wiki-su/translated")

In [22]:
from datasets import concatenate_datasets

id_cbn = concatenate_datasets([cbn_wiki_id, id_wiki_id])
jv_cbn = concatenate_datasets([cbn_wiki_jv, id_wiki_jv])
su_cbn = concatenate_datasets([cbn_wiki_su, id_wiki_su])

In [25]:
len(su_cbn)

28551

In [26]:
id_cbn.save_to_disk("dataset/paralel_id_cbn_127k")
jv_cbn.save_to_disk("dataset/paralel_jv_cbn_24k")
su_cbn.save_to_disk("dataset/paralel_su_cbn_28k")

Saving the dataset (1/1 shards): 100%|██████████| 28551/28551 [00:00<00:00, 198725.47 examples/s]


In [27]:
paralel_bali_dict = load_from_disk("dataset/paralel_dataset_from_bali_dict")  

In [29]:
paralel_bali_dict = paralel_bali_dict.remove_columns(["prompt"])

In [31]:
paralel_60k = load_from_disk("dataset/paralel_dataset_60k")

In [33]:
def convert_dataset(data):
    data["id"] = data["custom_id"]
    return data

In [35]:
paralel_60k = paralel_60k.map(convert_dataset, remove_columns=["custom_id", "text", "prompt_text"])

Map: 100%|██████████| 59257/59257 [00:08<00:00, 7163.49 examples/s] 


In [39]:
paralel_300k = load_from_disk("dataset/paralel_dataset_300k")

In [40]:
paralel_300k

Dataset({
    features: ['id', 'cirebonese', 'indonesian', 'balinese'],
    num_rows: 296203
})

In [41]:
paralel_bali_dict

Dataset({
    features: ['id', 'balinese', 'indonesian', 'cirebonese'],
    num_rows: 22248
})

In [42]:
paralel_60k

Dataset({
    features: ['indonesian', 'cirebonese', 'balinese', 'id'],
    num_rows: 59257
})

In [43]:
296203 + 22248 + 59257

377708

In [44]:
paralel_377k = concatenate_datasets([paralel_300k, paralel_60k, paralel_bali_dict])

In [45]:
paralel_377k.save_to_disk("dataset/paralel_3_lang/paralel_dataset_377k")

Saving the dataset (4/4 shards): 100%|██████████| 377708/377708 [00:04<00:00, 82903.84 examples/s] 


In [52]:
id_cbn = load_from_disk("dataset/paralel_2_lang/paralel_id_cbn_127k_dedup")

In [53]:
existing_id_text = set(paralel_377k["indonesian"])

In [54]:
def filter_id_cbn(data):
    return data["text"] not in existing_id_text

In [55]:
filtered_id_cbn = id_cbn.filter(filter_id_cbn)

Filter: 100%|██████████| 92504/92504 [00:13<00:00, 6913.87 examples/s]


In [57]:
filtered_id_cbn.save_to_disk("dataset/paralel_2_lang/paralel_id_cbn_127k_filtered")

Saving the dataset (1/1 shards): 100%|██████████| 44743/44743 [00:00<00:00, 53675.53 examples/s]


In [78]:
from datasets import load_dataset

id_mmlu = load_dataset("indolem/IndoMMLU")

In [79]:
bali_mmlu = id_mmlu["test"].filter(lambda x: x["subject"] == "Balinese")

In [80]:
bali_mmlu[0]

{'subject': 'Balinese',
 'group': 'Local languages and cultures',
 'level': 'SD',
 'class': '2',
 'question': 'Dugas Galungane titiang mabakti nganggon udeng baru. Udeng basa Bali alusne...',
 'options': "['A. destar', 'B. kampuh', 'C. wastra']",
 'answer': 'A',
 'is_for_fewshot': '0'}

In [81]:
import ast
import uuid

def transform_data(item):
    question_id = str(uuid.uuid4())

    # Parse the options string into a list
    options_str = item['options']
    options_list = ast.literal_eval(options_str)
    
    # Extract labels and texts from options
    labels = []
    texts = []
    for opt in options_list:
        # Split by dot and strip whitespace
        parts = opt.split('.', 1)
        labels.append(parts[0].strip().lower())  # Get 'A', 'B', 'C' and convert to lowercase
        texts.append(parts[1].strip())  # Get the actual option text
    
    # Create the transformed dictionary
    transformed = {
        "context": "",
        "question": item['question'],
        "choices": {
            "label": labels,
            "text": texts
        },
        "answer": item['answer'].lower(),  # Convert answer to lowercase to match labels
        "category": [],
        "grade": int(item['class']),
        "question_id": question_id
    }
    
    return transformed

In [82]:
bali_mmlu = bali_mmlu.map(transform_data, remove_columns=['options', 'class', 'subject', 'group', 'level', 'class', 'is_for_fewshot'])

Map: 100%|██████████| 471/471 [00:00<00:00, 7187.59 examples/s]


In [83]:
bali_mmlu[0]

{'question': 'Dugas Galungane titiang mabakti nganggon udeng baru. Udeng basa Bali alusne...',
 'answer': 'a',
 'context': '',
 'choices': {'label': ['a', 'b', 'c'], 'text': ['destar', 'kampuh', 'wastra']},
 'category': [],
 'grade': 2,
 'question_id': 'e22caaf5-7596-4d76-a04e-61637f60a8c1'}

In [84]:
bali_mmlu.save_to_disk("dataset/bali_mmlu")

Saving the dataset (1/1 shards): 100%|██████████| 471/471 [00:00<00:00, 60177.81 examples/s]


In [85]:
paralel_377k = load_from_disk("dataset/paralel_3_lang/paralel_dataset_377k_dedup")

In [86]:
paralel_377k

Dataset({
    features: ['id', 'cirebonese', 'indonesian', 'balinese'],
    num_rows: 348695
})

In [87]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o-mini")

def num_tokens_from_string(string: str, encoder) -> int:
    """Returns the number of tokens in a text string."""
    num_tokens = len(encoder.encode(string))
    return num_tokens

def count_tokens_in_dataset(dataset, field_name, num_tokens_from_string, encoder):
    total_tokens = 0
    
    for item in dataset:
        text = item[field_name]
        tokens = num_tokens_from_string(text, encoder)
        total_tokens += tokens
    
    return total_tokens

In [88]:
print(count_tokens_in_dataset(paralel_377k, "indonesian", num_tokens_from_string, encoding))

144553459


In [89]:
print(count_tokens_in_dataset(paralel_377k, "cirebonese", num_tokens_from_string, encoding))

156025125


In [90]:
print(count_tokens_in_dataset(paralel_377k, "balinese", num_tokens_from_string, encoding))

158701431
